# Allgemine Analyse

*Kai, Andrew und Fabian*

---

## 1. Bestimmung der Integrationsordnung

- Augmented Dickey-Fuller Test (ADF-Test)
- Phillips-Perron Test (PP-Test)
- Kwiatkowski-Phillips-Schmidt-Shin Test (KPSS-Test)


## 2. Transformation zur Stationarität

- Differenzierung
- Erste Differenz: $y_t - y_{t-1}$
- Zweite Differenz: $(y_t - y_{t-1}) - (y_{t-1} - y_{t-2})$
- Logarithmische Transformation
- Moving Average
- Simple Exponential Smoothing
- HP-Filter (Hodrick-Prescott-Filter)


## 3. ACF und PACF Analyse

- Autokorrelationsfunktion (ACF)
- Partielle Autokorrelationsfunktion (PACF)


## 4. Modellspezifikation

- AR-Modelle (Autoregressive)
- MA-Modelle (Moving Average)
- ARMA-Modelle
- ARIMA-Modelle
- Auto-ARIMA-Modell